In [1]:
import yfinance as yf

In [3]:
data = yf.download("RELIANCE.NS", start="2025-01-01", end="2025-12-31")

print(data.head())

/tmp/ipykernel_12040/2184721999.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("RELIANCE.NS", start="2025-01-01", end="2025-12-31")
[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open      Volume
Ticker      RELIANCE.NS  RELIANCE.NS  RELIANCE.NS  RELIANCE.NS RELIANCE.NS
Date                                                                      
2025-01-01  1221.250000  1226.300049  1211.599976  1214.849976     5892590
2025-01-02  1241.800049  1244.449951  1220.000000  1221.250000    15486276
2025-01-03  1251.150024  1262.050049  1235.500000  1243.900024    15521102
2025-01-06  1218.000000  1262.000000  1215.000000  1253.949951    14816766
2025-01-07  1240.849976  1244.500000  1221.250000  1222.000000    10070505


from nsetools import nse

In [18]:
from nsetools import Nse
nse = Nse()

all_stock_codes = nse.get_stock_codes()
print('Total Stock Codes: '+ str(len(all_stock_codes)))
# Print first 10 tickers
for code in list(all_stock_codes)[1:11]:
    print(code)


Total Stock Codes: 2101
21STCENMGM
360ONE
3IINFOLTD
3MINDIA
3PLAND
5PAISA
63MOONS
A2ZINFRA
AAATECH
AADHARHFC


In [16]:
tickers = [code + ".NS" for code in list(all_stock_codes)[1:]]


In [19]:
#list all the tickers to a .csv file
from nsetools import Nse
import pandas as pd

nse = Nse()

# Get all stock codes
all_stock_codes = nse.get_stock_codes()

# Skip the first row (header-like entry)
tickers = list(all_stock_codes)[1:]

# Add ".NS" for yfinance compatibility
tickers_with_suffix = [ticker + ".NS" for ticker in tickers]

# Save to CSV
df = pd.DataFrame(tickers_with_suffix, columns=["Ticker"])
df.to_csv("nse_tickers.csv", index=False)

print(f"✅ Saved {len(tickers)} NSE tickers to 'nse_tickers.csv'")


✅ Saved 2100 NSE tickers to 'nse_tickers.csv'


In [ ]:
#identify top 50 performing stocks

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Load tickers from previous CSV
tickers = pd.read_csv("nse_tickers.csv")["Ticker"].tolist()

# Optional: limit for testing
# tickers = tickers[:200]  # Limit to first 200 tickers to avoid throttling

start = (datetime.today() - timedelta(days=180)).strftime("%Y-%m-%d")
end = datetime.today().strftime("%Y-%m-%d")

results = []

for ticker in tickers:
    try:
        data = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=True)
        if len(data) > 0:
            start_price = data["Close"].iloc[0]
            end_price = data["Close"].iloc[-1]
            pct_return = ((end_price - start_price) / start_price) * 100
            results.append({"Ticker": ticker, "Return %": pct_return})
    except Exception as e:
        print(f"⚠️ Error fetching {ticker}: {e}")

# Create dataframe and sort
df_returns = pd.DataFrame(results)
df_returns = df_returns.sort_values(by="Return %", ascending=False).reset_index(drop=True)

# Save top 50
df_top50 = df_returns.head(50)
df_top50.to_csv("top_50_performing_stocks.csv", index=False)

print("✅ Top 50 performing stocks saved to 'top_50_performing_stocks.csv'")



1 Failed download:
['AEROENTER.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-12-27 -> 2025-06-25)')


In [5]:
import requests
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0"
}

# NSE API URL (Live Variation)
url = "https://www.nseindia.com/api/live-analysis-variations?index=gainers"

# Create session for cookies
session = requests.Session()
session.headers.update(headers)
session.get("https://www.nseindia.com")  # initial hit for cookies

# Get JSON response
res = session.get(url)
data = res.json()

# Check available categories
available_categories = [cat[0] for cat in data['legends']]
print("Available Categories:", available_categories)

# Let's focus on NIFTY only for now
nifty_data = data.get("NIFTY", {}).get("data", [])

# Convert to DataFrame
df = pd.DataFrame(nifty_data)

# Rename and sort
df = df.rename(columns={
    "symbol": "Symbol",
    "ltp": "Last Price",
    "prev_price": "Previous Price",
    "perChange": "% Change",
    "trade_quantity": "Volume",
    "turnover": "Turnover (Cr)"
})

df = df.sort_values(by="% Change", ascending=False).reset_index(drop=True)

# Keep top 50 if available
df_top50 = df.head(50)

# Save to CSV
df_top50.to_csv("nifty_top_50_gainers.csv", index=False)

print("✅ Top 50 NIFTY gainers saved to 'nifty_top_50_gainers.csv'")


Available Categories: ['NIFTY', 'BANKNIFTY', 'NIFTYNEXT50', 'SecGtr20', 'SecLwr20', 'FOSec', 'allSec']
✅ Top 50 NIFTY gainers saved to 'nifty_top_50_gainers.csv'


In [6]:
display(df_top50)

,Symbol,series,open_price,high_price,low_price,Last Price,Previous Price,net_price,Volume,Turnover (Cr),market_type,ca_ex_dt,ca_purpose,% Change
0,JIOFIN,EQ,304.80,312.40,304.10,309.00,303.30,1.88,21789411,67579.858216,N,-,-,1.88
1,BAJFINANCE,EQ,930.00,953.50,928.50,941.00,928.00,1.40,4728009,44643.752182,N,16-Jun-2025,Bonus 4:1,1.40
2,ETERNAL,EQ,259.42,263.25,258.95,262.11,258.86,1.26,17032432,44635.191299,N,-,-,1.26
3,BHARTIARTL,EQ,1970.00,2009.90,1966.30,1987.60,1965.60,1.12,3261870,64938.612708,N,07-Aug-2024,Dividend - Rs 8 Per Share,1.12
4,RELIANCE,EQ,1469.10,1488.00,1465.10,1483.70,1467.30,1.12,4431156,65563.384176,N,28-Oct-2024,Bonus 1:1,1.12
5,ULTRACEMCO,EQ,11690.00,11986.00,11681.00,11855.00,11733.00,1.04,250186,29697.478498,N,30-Jul-2024,Annual General Meeting/Dividend - Rs 70 Per Share,1.04
6,HINDALCO,EQ,674.10,683.80,672.70,679.60,672.65,1.03,2662118,18085.364845,N,09-Aug-2024,Annual General Meeting/Dividend - Rs 3.5 Per S...,1.03
7,ADANIPORTS,EQ,1399.00,1409.70,1395.20,1406.10,1393.10,0.93,635517,8926.154024,N,13-Jun-2025,Dividend - Rs 7 Per Share,0.93
8,GRASIM,EQ,2840.00,2889.60,2832.00,2856.30,2831.80,0.87,681426,19524.626608,N,06-Aug-2024,Annual General Meeting/Dividend - Rs 10 Per Share,0.87
9,HDFCBANK,EQ,1981.40,1999.90,1980.00,1995.30,1980.20,0.76,4555935,90695.453639,N,27-Jun-2025,Dividend - Rs 22 Per Share,0.76


In [9]:
import time
start = time.time()
#df = pd.read_csv("https://www1.nseindia.com/content/indices/ind_nifty500list.csv")
df = pd.read_csv("https://archives.nseindia.com/content/indices/ind_nifty500list.csv")

end = time.time()
print(f"✅ Downloaded in {end - start:.2f} seconds")


✅ Downloaded in 0.35 seconds
